In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

In [2]:
df_movies = pd.read_csv("Data/movies.csv")
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df_ratings = pd.read_csv("Data/ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df_ratings.drop(columns='timestamp', inplace=True)

In [5]:
df_cleaned = df_movies.merge(df_ratings, on='movieId')

In [6]:
df_cleaned.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [7]:
df_cleaned['num_viewers'] = df_cleaned.groupby('movieId')['userId'].transform('count')


In [8]:
df_cleaned.head()

,movieId,title,genres,userId,rating,num_viewers
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,215
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,215
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,215


In [9]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      100836 non-null  int64  
 1   title        100836 non-null  object 
 2   genres       100836 non-null  object 
 3   userId       100836 non-null  int64  
 4   rating       100836 non-null  float64
 5   num_viewers  100836 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [10]:
df_clean = df_cleaned.drop_duplicates(subset='title', keep='first')
df_clean.head()

,movieId,title,genres,userId,rating,num_viewers
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215
215,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,110
325,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,52
377,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6,3.0,7
384,5,Father of the Bride Part II (1995),Comedy,6,5.0,49


In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9719 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movieId      9719 non-null   int64  
 1   title        9719 non-null   object 
 2   genres       9719 non-null   object 
 3   userId       9719 non-null   int64  
 4   rating       9719 non-null   float64
 5   num_viewers  9719 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 531.5+ KB


In [12]:
genres_split = df_clean.genres.apply(lambda x: x.split(sep='|')).apply(pd.value_counts, 1.0).fillna(0.0)
df_clean = pd.concat([df_clean.iloc[:,:], genres_split], axis=1)
df_clean.head()


,movieId,title,genres,userId,rating,num_viewers,Children,Comedy,Fantasy,Adventure,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,110,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,52,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
377,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6,3.0,7,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384,5,Father of the Bride Part II (1995),Comedy,6,5.0,49,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9719 entries, 0 to 100835
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             9719 non-null   int64  
 1   title               9719 non-null   object 
 2   genres              9719 non-null   object 
 3   userId              9719 non-null   int64  
 4   rating              9719 non-null   float64
 5   num_viewers         9719 non-null   int64  
 6   Children            9719 non-null   float64
 7   Comedy              9719 non-null   float64
 8   Fantasy             9719 non-null   float64
 9   Adventure           9719 non-null   float64
 10  Animation           9719 non-null   float64
 11  Romance             9719 non-null   float64
 12  Drama               9719 non-null   float64
 13  Action              9719 non-null   float64
 14  Thriller            9719 non-null   float64
 15  Crime               9719 non-null   float64
 16  Horr

In [14]:
df_clean['(no genres listed)'].sum()

34.0

In [15]:
df_clean.drop(columns=['(no genres listed)'], inplace=True)

In [16]:
df_clean.head()

,movieId,title,genres,userId,rating,num_viewers,Children,Comedy,Fantasy,Adventure,...,Crime,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,110,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,52,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
377,4,Waiting to Exhale (1995),Comedy|Drama|Romance,6,3.0,7,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384,5,Father of the Bride Part II (1995),Comedy,6,5.0,49,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Hybrid Model

In [17]:
import pandas as pd
import surprise
from surprise import accuracy
from surprise import Dataset, Reader, SVD, KNNWithMeans
from surprise.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [18]:
# create a reader to read the dataframe
reader = Reader(rating_scale=(1.0, 5.0))

# create a surprise dataset from the dataframe
data = Dataset.load_from_df(df_clean[['userId', 'movieId', 'rating']], reader)

# split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# train a collaborative filtering model using SVD
algo_cf = SVD()
algo_cf.fit(trainset)

# train a content-based model using the KNN algorithm
k = 10
sim_options = {'name': 'cosine', 'user_based': False}
algo_cb = KNNWithMeans(k=k, sim_options=sim_options)
algo_cb.fit(trainset)

# for each user and item pair in the test set, make predictions using both models
predictions_cf = algo_cf.test(testset)
predictions_cb = algo_cb.test(testset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
# assign weights to the models
weight_cf = 0.5
weight_cb = 0.5

# combine the predictions from both models using a weighted average
predictions_combined = []

# iterate over each prediction in the test set
for pred_cf, pred_cb in zip(predictions_cf, predictions_cb):
    # extract the user ID and item ID from the prediction
    uid, iid = pred_cf.uid, pred_cf.iid
    
    # calculate the weighted sum of the predicted ratings
    rating_combined = (weight_cf * pred_cf.est) + (weight_cb * pred_cb.est)
    
    # create a new prediction object with the combined rating
    pred_combined = surprise.prediction_algorithms.predictions.Prediction(uid, iid, r_ui=None, est=rating_combined, details=None)
    
    # add the combined prediction to the list
    predictions_combined.append(pred_combined)


# Recommendation for Top 5

In [33]:
# fit the nearest neighbors model
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(df_clean[['rating']])

NearestNeighbors(algorithm='brute', metric='cosine')

In [54]:
#Recommndation Model



In [57]:
recommend_movies('Jumanji (1995)')

,title,mean,count
0,"Breed, The (2006)",5.0,1
3,Ocean's Thirteen (2007),4.5,1


In [51]:
# create a new DataFrame with movie title, average rating, and number of viewers
movie_stats = df_clean.groupby('title').agg({'rating': [np.size, np.mean]})

# select the 5 movies with the highest average rating
top_movies = movie_stats['rating']['mean'].nlargest(5)

# select the movie titles of the top 5 movies
top_movie_titles = top_movies.index.tolist()

# filter the dataframe to only include the top 5 movies
top_movie_df = df_clean[df_clean['title'].isin(top_movie_titles)]

# group the data by movie title and rating, and count the number of viewers for each rating
top_movie_grouped = top_movie_df.groupby(['title', 'rating']).agg({'userId': 'count'}).reset_index()

# print the top 5 movies with their average rating, number of viewers, and each rating
for movie_title in top_movie_titles:
    print(f'Top 5 ratings for {movie_title}:')
    print(top_movie_grouped[top_movie_grouped['title'] == movie_title])

Top 5 ratings for 'Salem's Lot (2004):
                 title  rating  userId
0  'Salem's Lot (2004)     5.0       1
Top 5 ratings for 'burbs, The (1989):
                title  rating  userId
1  'burbs, The (1989)     5.0       1
Top 5 ratings for 10 Cloverfield Lane (2016):
                        title  rating  userId
2  10 Cloverfield Lane (2016)     5.0       1
Top 5 ratings for 10 Things I Hate About You (1999):
                               title  rating  userId
3  10 Things I Hate About You (1999)     5.0       1
Top 5 ratings for 12 Angry Men (1957):
                 title  rating  userId
4  12 Angry Men (1957)     5.0       1


In [ ]:
#########################

PLOTS BELOW

PLOTS ABOVE

# Extra Code for Model Tuning 

In [ ]:
import surprise
from surprise import Dataset, Reader, SVD
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset using Surprise
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(df_clean[['userId', 'title', 'rating']], reader)

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
from surprise import SVD

algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

In [ ]:
#algo.test(test)

In [ ]:
surprise.accuracy.mae(algo.test(test))

In [ ]:
surprise.accuracy.rmse(algo.test(test))

In [ ]:
from surprise.prediction_algorithms.matrix_factorization import SVD
SVD = SVD()

In [ ]:
from surprise.model_selection import RandomizedSearchCV

# Define the search space for hyperparameters
param_distributions = {'n_factors': [50, 100, 200],
                       'n_epochs': [10, 20, 30],
                       'lr_all': [0.002, 0.005, 0.01],
                       'reg_all': [0.02, 0.1, 0.4]}
# Create the randomized search object
rs = RandomizedSearchCV(SVD, param_distributions=, n_iter=10, measures=['rmse', 'mae'], cv=5)

# Run the randomized search
rs.fit(data)

# Get the best RMSE score and the corresponding hyperparameters
#best_rmse = rs.best_score['rmse']
#best_params = rs.best_params['rmse']


In [ ]:
print('Best RMSE: ' + str(best_rmse))

In [ ]:
print('Best Params: ' + str(best_params))